In [1]:
from __future__ import division
from numpy.random import randn
from pandas import Series
import numpy as np
import pandas as pd
from datetime import datetime
import math
np.set_printoptions(precision=4)
import sys; sys.path.append('book_scripts')
%pwd

u'E:\\project\\pychram\\book_scripts'

In [2]:
def load_stock_base_msg():
    stock_data_path = 'E:/project/pychram/traderesp/base/output-csv/'
    stock_data_cvs = stock_data_path + 'output-2013-2014-all-stock-base-msg.csv'
    #
    stat = pd.read_csv(stock_data_cvs,encoding='gbk',index_col=['code'])
    return stat

In [3]:
def get_date_period_staticstics(begtime,endtime):
    #
    stock_data_path = 'E:/project/pychram/traderesp/strategy/output-csv/'
    stock_data_cvs = stock_data_path + 'output-stock-history-adjust-m26-kdj-' \
                                     + begtime.strftime('%Y%m%d') + '-' + endtime.strftime('%Y%m%d') + '.csv'
    #
    stat = pd.read_csv(stock_data_cvs,parse_dates=['buy_date','sell_date'],encoding='gbk')
    
    #逐笔利润
    stat['profit'] = stat['sell_price'] - stat['buy_price']
    #
    action_num = len(stat)
    suc_num = stat[stat['profit'] > 0.00 ].count() 
    fail_num = action_num - suc_num
    #
    tmp_profit = stat.groupby(['code']).sum()
    year_action_num = len(tmp_profit)
    tmp_profit['profit_rate'] = (tmp_profit['sell_price'] - tmp_profit['buy_price']) / tmp_profit['buy_price']
    year_suc_num = tmp_profit[tmp_profit['profit_rate'] > 0.00].count()
    year_fail_num = year_action_num - year_suc_num
    #
    year_suc_num_gt_percet_5 = tmp_profit[tmp_profit['profit_rate'] > 0.05].count()
    #
    stat_sum_all = stat.sum()
    code_unique_num = len(stat['code'].unique())
    s_ge_b_num = stat[stat['sell_price'] > stat['buy_price']].count()
    #
    print '-----  ',begtime,'  ----  ',endtime,'  -----'
    print '股票总数(排重) :',year_action_num
    print '买卖总次数(一次买卖算一次) :',action_num
    print '其中获利次数 :',suc_num['profit']
    print '其中亏损次数 :',fail_num['profit']
    print '获利次数比率 :',suc_num['profit'] / action_num
    print '亏损次数比率 :',fail_num['profit'] / action_num
    print '年度获利股票数 :',year_suc_num['profit_rate']
    print '年度亏损股票数 :',year_fail_num['profit_rate']
    print '年度获利股票数比率 :',year_suc_num['profit_rate'] / year_action_num
    print '年度亏损股票数比率 :',year_fail_num['profit_rate'] / year_action_num
    print '年度获利股票数( 盈利大于5%) :',year_suc_num_gt_percet_5['profit_rate']
    print '年度获利股票数比率( 盈利大于5%) :',year_suc_num_gt_percet_5['profit_rate'] / year_action_num
    print '---------------------------------------------------------------------'
    #
    return tmp_profit[tmp_profit['profit_rate'] > 0.00].sort_values('profit_rate',ascending=False),tmp_profit,stat

In [4]:
def show_date_staticstics():
    code1,code1_tmp1,code1_tmp2 = get_date_period_staticstics(datetime(2013,1,1),datetime(2013,12,31))
    code2,code2_tmp1,code2_tmp2 = get_date_period_staticstics(datetime(2014,1,1),datetime(2014,12,31))
    code3 = set(code1.index) & set(code2.index)
    year_2_num = len(code3)
    print '连续2年上涨股票数(获利股票数) :',year_2_num
    print '连续2年上涨股票数比率 :',year_2_num / len(code1) if len(code1) > 0 else 0
    #--------------------------------------------------------
    code1_gt_percent_5 =  code1[code1['profit_rate'] > 0.05]
    code2_gt_percent_5 =  code2[code2['profit_rate'] > 0.05]
    code3_gt_percent_5 =  set(code1_gt_percent_5.index) & set(code2_gt_percent_5.index)
    print '连续2年上涨股票数(获利股票数 大于 5%) :',len(code3_gt_percent_5)
    print '连续2年上涨股票数比率(获利大于 5%) :',len(code3_gt_percent_5) / len(code1) if len(code1) > 0 else 0
    #--------------------------------------------------------
    code4 = code1.index[0:100]
    code5 = code2.index[0:100]
    code6 = set(code4) & set(code5)
    year_2_num_from_100 = len(code6)
    print '连续2年上涨股票数(Top 100) :',year_2_num_from_100
    code7 = code1.index[0:200]
    code8 = code2.index[0:200]
    code9 = set(code7) & set(code8)
    year_2_num_from_200 = len(code9)
    print '连续2年上涨股票数(Top 200) :',year_2_num_from_200
    code10 = code1.index[0:300]
    code11 = code2.index[0:300]
    code12 = set(code10) & set(code11)
    year_2_num_from_300 = len(code12)
    print '连续2年上涨股票数(Top 300) :',year_2_num_from_300
    code13 = code1.index[0:400]
    code14 = code2.index[0:400]
    code15 = set(code13) & set(code14)
    year_2_num_from_400 = len(code15)
    print '连续2年上涨股票数(Top 400) :',year_2_num_from_400
    code16 = code1.index[0:500]
    code17 = code2.index[0:500]
    code18 = set(code16) & set(code17)
    year_2_num_from_500 = len(code18)
    print '连续2年上涨股票数(Top 500) :',year_2_num_from_500
    #
    code_bing = pd.merge(code1,code2,left_index=True, right_index=True)
    return code_bing,code_bing[(code_bing['profit_rate_x'] > 0.05) & (code_bing['profit_rate_y'] > 0.05)]

In [5]:
code_base_msg = load_stock_base_msg()
code_profit_rate_ge_0, code_profit_rate_ge_5 = show_date_staticstics()
code_profit_rate_ge_5 = code_profit_rate_ge_5[['profit_rate_x','profit_rate_y']]

-----   2013-01-01 00:00:00   ----   2013-12-31 00:00:00   -----
股票总数(排重) : 2468
买卖总次数(一次买卖算一次) : 68915
其中获利次数 : 31287
其中亏损次数 : 37628
获利次数比率 : 0.453994050642
亏损次数比率 : 0.546005949358
年度获利股票数 : 950
年度亏损股票数 : 1518
年度获利股票数比率 : 0.384927066451
年度亏损股票数比率 : 0.615072933549
年度获利股票数( 盈利大于5%) : 1
年度获利股票数比率( 盈利大于5%) : 0.000405186385737
---------------------------------------------------------------------
-----   2014-01-01 00:00:00   ----   2014-12-31 00:00:00   -----
股票总数(排重) : 2592
买卖总次数(一次买卖算一次) : 106587
其中获利次数 : 56329
其中亏损次数 : 50258
获利次数比率 : 0.528479082815
亏损次数比率 : 0.471520917185
年度获利股票数 : 2043
年度亏损股票数 : 549
年度获利股票数比率 : 0.788194444444
年度亏损股票数比率 : 0.211805555556
年度获利股票数( 盈利大于5%) : 22
年度获利股票数比率( 盈利大于5%) : 0.00848765432099
---------------------------------------------------------------------
连续2年上涨股票数(获利股票数) : 705
连续2年上涨股票数比率 : 0.742105263158
连续2年上涨股票数(获利股票数 大于 5%) : 0
连续2年上涨股票数比率(获利大于 5%) : 0.0
连续2年上涨股票数(Top 100) : 0
连续2年上涨股票数(Top 200) : 3
连续2年上涨股票数(Top 300) : 11
连续2年上涨股票数(Top 400) : 31
连续2年上涨股票数

In [6]:
code_bing = pd.merge(code_base_msg,code_profit_rate_ge_5,left_index=True, right_index=True)

In [7]:
code_bing.columns

Index([    u'code_name',       u'sina_hy',       u'sina_gn',       u'sina_dy',
       u'profit_rate_x', u'profit_rate_y'],
      dtype='object')

In [8]:
def trim_float_num(x):
    #print type(x)
    #print 'x=',x
    xx = 0.000
    if x is None:
        xx = 0.000
    else:
        xx = float(str(x))
    # print xx
    if xx < 0.001:
        return "0.1 %"
    else:
        ret = '%.1f' %(xx * 100.00)
        return ret + ' %'

In [9]:
trim_profit_rate_x = code_bing['profit_rate_x'].map(trim_float_num)
code_bing['profit_rate_x'] = trim_profit_rate_x
trim_profit_rate_y = code_bing['profit_rate_y'].map(trim_float_num)
code_bing['profit_rate_y'] = trim_profit_rate_y

In [10]:
code_bing

,code_name,sina_hy,sina_gn,sina_dy,profit_rate_x,profit_rate_y
code,,,,,,


In [11]:
code1,code_2,code_all = get_date_period_staticstics(datetime(2013,1,1),datetime(2013,12,31))

-----   2013-01-01 00:00:00   ----   2013-12-31 00:00:00   -----
股票总数(排重) : 2468
买卖总次数(一次买卖算一次) : 68915
其中获利次数 : 31287
其中亏损次数 : 37628
获利次数比率 : 0.453994050642
亏损次数比率 : 0.546005949358
年度获利股票数 : 950
年度亏损股票数 : 1518
年度获利股票数比率 : 0.384927066451
年度亏损股票数比率 : 0.615072933549
年度获利股票数( 盈利大于5%) : 1
年度获利股票数比率( 盈利大于5%) : 0.000405186385737
---------------------------------------------------------------------


In [1]:
code_all[code_all['code']==858]

NameError: name 'code_all' is not defined

In [13]:
code2,code21,code2_all = get_date_period_staticstics(datetime(2014,1,1),datetime(2014,12,31))

-----   2014-01-01 00:00:00   ----   2014-12-31 00:00:00   -----
股票总数(排重) : 2592
买卖总次数(一次买卖算一次) : 106587
其中获利次数 : 56329
其中亏损次数 : 50258
获利次数比率 : 0.528479082815
亏损次数比率 : 0.471520917185
年度获利股票数 : 2043
年度亏损股票数 : 549
年度获利股票数比率 : 0.788194444444
年度亏损股票数比率 : 0.211805555556
年度获利股票数( 盈利大于5%) : 22
年度获利股票数比率( 盈利大于5%) : 0.00848765432099
---------------------------------------------------------------------


In [14]:
code2_all[code2_all['code']==858]

,buy_date,buy_money,buy_num,buy_price,code,sell_date,sell_money,sell_num,sell_price,profit
1332,2014-02-11,23272.180995,100,232.721810,858,2014-02-17,25186.873837,100,251.868738,19.146928
4402,2014-02-18,24968.931818,100,249.689318,858,2014-02-24,24533.083155,100,245.330832,-4.358487
5736,2014-02-25,23521.240673,100,235.212407,858,2014-02-26,23505.669612,100,235.056696,-0.155711
7830,2014-03-04,24751.011316,100,247.510113,858,2014-03-05,25171.308239,100,251.713082,4.202969
11059,2014-03-25,26852.510274,100,268.525103,858,2014-03-31,25949.657771,100,259.496578,-9.028525
13107,2014-04-02,26899.220283,100,268.992203,858,2014-04-08,26805.840237,100,268.058402,-0.933800
15305,2014-04-09,26977.075944,100,269.770759,858,2014-04-14,27973.344060,100,279.733441,9.962681
17774,2014-04-15,27397.372906,100,273.973729,858,2014-04-17,27506.330482,100,275.063305,1.089576
19769,2014-04-18,27490.761899,100,274.907619,858,2014-04-22,26603.453464,100,266.034535,-8.873084
20956,2014-04-23,27553.010528,100,275.530105,858,2014-04-24,27817.657194,100,278.176572,2.646467


In [15]:
code21.ix[858]

buy_money      1360213.357812
buy_num           4700.000000
buy_price        13602.133578
sell_money     1369962.657971
sell_num          4700.000000
sell_price       13699.626580
profit              97.493002
profit_rate          0.007167
Name: 858, dtype: float64